
# ЛАБОРАТОРНА РОБОТА  

## "Лінійні та нелінійні регресійні моделі МН"

__Метою__ лабораторної роботи є набуття практичних навичок використання модулів бібліотеки `Scikit-learn` для вирішення наступних задач:

- визначення суттєвих показчиків для регресійної моделі
- пошук та настроювання гіперпараметрів лінійних та нелінійних регресійних моделей

__Результатом__ виконання лабораторної роботи є серія моделей які прогнозують ціну кватрир на вторинному ринку житла

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('apartment_transformed.csv')
data.shape

(483, 199)

In [3]:
# відокремити ціловий показчик 'price'
price = data['Price']

# зилишити в 'data' тільки незалежні показчики
data = data.drop(columns = ['Price'])

data.shape, price.shape

((483, 198), (483,))

## 1. Пошук значущих ознак

### Теоретичне введення

__Значущі (суттеві) ознаки__ це дані, які мають сильну кореляцію або вплив на результат або прогноз моделі.

Ці позазчики визначаються за допомогою процесу, який називається __відбором ознак__ ([Feature selection](https://en.wikipedia.org/wiki/Feature_selection)), який передбачає _оцінку_ та _ранжування важливості_ різних змінних у наборі даних. Це можна зробити за дпомогою статистичних тестів, кореляційного аналізу або алгоритмів машинного навчання.

Після визначення значущих ознак їх можна використовувати для навчання моделі машинного навчання.

Це може бути особливо важливим у моделях, де дані складні та містять багато змінних. Тому ідентифікація  суттєвих ознак може допомогти зменшити розмірність даних і покращити продуктивність моделі.

Класи в модулі [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html) можна використовувати для вибору функцій/зменшення розмірності на вибіркових наборах або для покращення показників точності оцінювачів, або для підвищення їх продуктивності на масивах даних з дуже великою розмірністю.

### Завдання

Відібрати з вхідного набору `data` 7 найбільш суттєвих показчиків для регрісійної моделі машинного навчання.

In [4]:
# імпортувати з модуля 'feature_selection' селектор ознак 'SelectKBest' 
# та регрісійний тест 'f_regression'
from sklearn.feature_selection import SelectKBest, f_regression

In [5]:
# побудувати селектор 7 ознак на f-регресорі
kbest_selector = SelectKBest(f_regression, k = 7)

# застосувати селектор для побудови списку ознак
data_selected = kbest_selector.fit_transform(data, price)

In [6]:
# зберегти імена визначених селектором найбільш суттєвих ознак
best_features = kbest_selector.get_feature_names_out()
best_features

array(['rooms', 'price_per_m2', 'area_total', 'area_comfort',
       'street_Мічуріна', 'street_Саксаганського',
       'district_Голосіївський, Печерський, Шевченківський'], dtype=object)

In [7]:
# побудувати датафрейм на визначених ознаках
data = pd.DataFrame(data=data_selected, columns=best_features)
data.head()

,rooms,price_per_m2,area_total,area_comfort,street_Мічуріна,street_Саксаганського,"district_Голосіївський, Печерський, Шевченківський"
0,-1.103985,-0.151460,-0.988970,-0.392341,0.0,0.0,0.0
1,-1.103985,-0.151460,-0.988970,-0.392341,0.0,0.0,0.0
2,-0.151798,-0.126981,-0.287685,-0.309069,0.0,0.0,0.0
3,-0.151798,-0.015636,-0.268205,-0.059251,0.0,0.0,0.0
4,-0.151798,-0.128100,-0.248725,-0.364584,0.0,0.0,0.0


### Висновки

_описати загальну статистичну характеристику отриманого датасети та зробити висновки щодо можливості його використання для подальшого аналізу_

Отриманий датасет містить стандартизовані значення показників квартир, включаючи кількість кімнат, ціну за метр квадратний, загальну площу, площу без кухні та кореляцію з окремими районами та вулицями. Можна зробити висновок, що отриманий датасет може бути використаний для подальшого аналізу.

## 2. Множинна лінійна регресія

### Теоретичне введення

__Множинна лінійна регресія__ — це статистичний метод, який використовується для встановлення зв’язку між _залежною_ (цільвою) змінною $\textbf y$ та _кількома_ незалежними змінними $\textbf [X]$.  

__Метою__ множинної лінійної регресії є знаходження найкращого лінійного зв’язку між залежною змінною та незалежними змінними, який виражається у вигляді рівняння:
$$y = b_0 + b_1 x_1 + b_2 x_2 + ... + b_n x_n$$

Найкращій зв'язок забезпечується знаходженням таких коєфіцієнтів $[B]$, що додають мінімум обраній метриці (MSE, MAE, ...)

### Завдання


Порахувати показчики якості моделі [лінйной множинної регресії](https://uk.mcfairbanks.com/719-multiple-regression-formula) на визначениx п.1 значущих ознаках датасету застосувавши [кросс-валідацію з __10__ сплітами](https://scikit-learn.org/stable/modules/cross_validation.html).

In [9]:
# імпортувати та побудувати лінійний регресор з параметрами за замовчанням
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(data, price)

LinearRegression()

In [10]:
# імпортувати крос-валідатор 'cross_validate' з модуля 'model_selection'
from sklearn.model_selection import cross_validate

In [11]:
# отримати результати крос-валідації по параметрам 'neg_mean_absolute_percentage_error' 
# та 'r2' на 10 сплітах передбачивши розрахунок на навчальному наборі 'return_train_score'
cv_results_mul = cross_validate(lr, data, price, cv = 10,
                         scoring = ('r2', 'neg_mean_absolute_percentage_error'),
                         return_train_score = True)

#### занести результати в датафрейм 'cv_results_mul ' наступного вигляду:

------------

|помилка тесту в %   |коєф. R2 тесту  | помилка навчання в %  | коєф. R2 навчання  |
| :------------:|:------------:|:------------:|:------------:|
|  xx.xx | xx.xx  | xx.xx  | xx.xx  |
|  xx.xx | xx.xx  | xx.xx  | xx.xx  |
|  ... | ...  | ...  | ...  |


In [12]:
cv_results_mul = pd.DataFrame({'помилка тесту в %': cv_results_mul['test_neg_mean_absolute_percentage_error'],
                              'коєф. R2 тесту': cv_results_mul['test_r2'],
                              'помилка навчання в %': cv_results_mul['train_neg_mean_absolute_percentage_error'],
                              'коєф. R2 навчання': cv_results_mul['train_r2']})
cv_results_mul.head()

,помилка тесту в %,коєф. R2 тесту,помилка навчання в %,коєф. R2 навчання
0,-0.496906,0.982911,-0.446099,0.939145
1,-0.384709,0.681537,-0.427471,0.986728
2,-0.393336,0.520854,-0.429380,0.986682
3,-0.455588,0.864394,-0.404060,0.987319
4,-0.348883,0.826308,-0.436310,0.987034


In [13]:
# продовжити наступні команди виводу:
print ("середня помилка навчання =", round(cv_results_mul['помилка навчання в %'].mean(), 2), "%")
print ("середня помилка тесту =", round(cv_results_mul['помилка тесту в %'].mean(), 2), "%")

середня помилка навчання = -0.42 %
середня помилка тесту = -0.44 %


### Висновки

_зпираючись на отримані метрики якості зробити висновок про придатність моделі, недонавчана чи перенавчана вона і т.п._

- Середня помилка навчання становить -0.42%, що можна вважати досить добрим показником;
- Середня помилка тесту становить -0.44%, що також є достатньо добрим результатом
- На основі отриманих метрик можна зробити висновок, що модель може бути достатньо придатною для прогнозування цін на нерухомість.

## 3. Гребнева (Ridge) регресія

### Теоретичне введення

__Гребнева регресія__ — це техніка регулярізації, яка використовується в машинному навчанні для запобігання перенавчанню лінійних регресійних моделей за рахунок додавання штрафу до функції втрат регресійної моделі, яка зменшує величину коефіцієнтів до нуля.

$$\min_w\sum_{i=1}^n(y_i - x_i^w)^2 + \lambda|w|_2^2$$

Розмір штрафу визначається [нормою L2](https://craftappmobile.com/l1-vs-l2-regularization/) вектора коефіцієнтів, помноженою на гіперпараметр $\large \lambda$.




### Завдання

Побудувати модель на основі `ridge-регресії` та за допомогою [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) знайти таке значення _L2-регулярізатора_, яке буде
мінізувати обрані метрики якості моделі.
Для побудови моделі скоистатися датасетом, що отримано в  __завданні 1__ лабораторної роботи


In [14]:
# імпортувати ridge-регресор з модуля `sklearn.linear_model`
from sklearn.linear_model import Ridge

# побудувати регресор
ridge = Ridge(random_state=42, fit_intercept=False)

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


In [15]:
# імпортувати сітку пошуку `GridSearchCV` з модулю sklearn.model_selection
from sklearn.model_selection import GridSearchCV

# визначити параметр равномірного пошуку 100 значень параметеру `alpha` в диапазоні 0-100000 
grid_params = {'alpha': [0, 100000],
               'max_iter': [0, 100]}

In [16]:
%%time

# побудувати та натренувати гребневу регресійну модель на сітці 'grid_params'
# в якості критерія оцінки якості взяти метрику `neg_mean_absolute_percentage_error`

# створюємо сітку пошуку та тренуємо на ній модель
grid_search_model = GridSearchCV(ridge, param_grid = grid_params,
                                 scoring='neg_mean_absolute_percentage_error')

grid_search_model.fit(data, price)

CPU times: total: 422 ms
Wall time: 174 ms


C:\Users\Katya\anaconda3\envs\ML\lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=3.91198e-19): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
C:\Users\Katya\anaconda3\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Katya\anaconda3\envs\ML\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Katya\anaconda3\envs\ML\lib\site-packa

GridSearchCV(estimator=Ridge(fit_intercept=False, random_state=42),
             param_grid={'alpha': [0, 100000], 'max_iter': [0, 100]},
             scoring='neg_mean_absolute_percentage_error')

In [30]:
# вивести найкращій естіматор (best_estimator_), та найкраще значення обраної метрики (best_score_)
grid_search_model.best_score_

-1.0136541192968491

In [18]:
grid_search_model.best_estimator_

Ridge(alpha=100000, fit_intercept=False, max_iter=100, random_state=42)

### Висновки

_cпираючись на отримані метрики якості зробити висновок про придатність моделі, недонавчана чи перенавчана вона і т.п._

З метрик, отриманих за допомогою grid_search, ми бачимо, що найкращим значенням метрики було -1.0136541192968491, що є досить близьким до 0, що може свідчити про те, що модель показує добрі результати. Однак, оскільки ми використовували Ridge регресію зі значенням alpha = 100000, можна припустити, що модель є недоналаштованою, оскільки таке велике значення alpha може призвести до високої помилки на тренувальних даних і недостатньої точності на тестових даних. Таким чином, можна сказати, що модель не є перенавченою, але може бути покращена.

## 4. Поліноміальна регресія

### Теоретичне введення

__Поліноміальна регресія__ — це тип нелінійної регресії, у якому зв’язок між незалежною змінною $\large x$ і залежною змінною $\large y$ моделюється як поліноміальна функція n-го ступеня:

$$ y = \beta_0 + \beta_1 x + \beta_2 x^2 + \beta_3 x^3 + \cdots + \beta_n x^n + \varepsilon $$

__Метою__ поліноміальної регресії є знаходження значень коефіцієнтів $ β_i$, які найкраще відповідають даним.

### Завдання


Порахувати показчики якості моделі на [поліноміальній регресії](https://uk.wikipedia.org/wiki/Поліноміальна_регресія) на визначених в п.1 значущих ознаках датасету, попередньо розширивши датасет за допомогою трансформера [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

In [19]:
# імпортувати модуль preprocessing.PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures

# побудувати трансформер ступеня 2 для побудови додаткових ознак в датасеті
poly = PolynomialFeatures(include_bias = False)
data_poly = poly.fit_transform(data)

In [20]:
# визначити імена відібраних показчиків
poly_features_names = poly.get_feature_names_out()

In [21]:
# побудувати датасет на визначеному поліномі `poly`
data_poly = pd.DataFrame(data_poly, columns = poly_features_names)
data_poly.shape

(483, 35)

In [22]:
# отримати результати крос-валідації на множинном регресорі `lr` по параметрам 'neg_mean_absolute_percentage_error' 
# та 'r2' на 10 сплітах передбачивши розрахунок на навчальному наборі 'return_train_score'
cv_results_poly = cross_validate(lr, data, price, cv = 10,
                         scoring = ('r2', 'neg_mean_absolute_percentage_error'),
                         return_train_score = True)

In [23]:
# занести результати крос-валідації: помилка тесту, помилка навчання та відповідні коефіцієнти 
# детермінаційї в датафрейм `cv_results_poly`. 
cv_results_poly = pd.DataFrame({'помилка тесту в %': cv_results_poly['test_neg_mean_absolute_percentage_error'],
                              'коєф. R2 тесту': cv_results_poly['test_r2'],
                              'помилка навчання в %': cv_results_poly['train_neg_mean_absolute_percentage_error'],
                              'коєф. R2 навчання': cv_results_poly['train_r2']})

cv_results_poly.head()

,помилка тесту в %,коєф. R2 тесту,помилка навчання в %,коєф. R2 навчання
0,-0.496906,0.982911,-0.446099,0.939145
1,-0.384709,0.681537,-0.427471,0.986728
2,-0.393336,0.520854,-0.429380,0.986682
3,-0.455588,0.864394,-0.404060,0.987319
4,-0.348883,0.826308,-0.436310,0.987034


In [24]:
# за допомогою крос-валідатора 'cross_val_predict' побудувати прогноз 'price_pred' 
# на лінійному регресорі на 10 сплітах
from sklearn.model_selection import cross_val_predict

price_pred = cross_val_predict(lr, data, price, cv = 10)

In [25]:
# вивести порівняльну таблицю з двох колонок: ціна реальна, ціна прогнозна
pred = pd.DataFrame({'ціна реальна': price,
                     'ціна прогнозна': price_pred})
pred['достовірність.'] = (pred['ціна прогнозна'] / pred['ціна реальна']).astype(float).round(3)
pred.head(5)

,ціна реальна,ціна прогнозна,достовірність.
0,30970.0,-54510.457275,-1.760
1,30970.0,-54510.457275,-1.760
2,82000.0,64389.479920,0.785
3,135000.0,155349.127598,1.151
4,84000.0,70239.194441,0.836


In [26]:
# натренувати регресор `lr` на поліноміальних ознаках `data_poly`
lr.fit(data_poly, price)

LinearRegression()

In [31]:
# сформувати таблицю коєфіцієнтів поліному
coef = pd.DataFrame({'Ознаки': poly_features_names,
                     'коеф.регресора': lr.coef_.astype('int')})
coef

,Ознаки,коеф.регресора
0,rooms,13273
1,price_per_m2,599130
2,area_total,96411
3,area_comfort,-3014
4,street_Мічуріна,49
5,street_Саксаганського,-1166
6,"district_Голосіївський, Печерський, Шевченківс...",-1166
7,rooms^2,1896
8,rooms price_per_m2,21777
9,rooms area_total,-7070


### Висновки

_Базуючись на значенях метрик абсолютної помилки та r2-оцінки, сформулювати вашу думку чи відповідає поліноміальна модель вимогам якості та дати характеристику декільком коефіцієнтам (3-4) на свій вибір._ 

- За результатами аналізу метрик абсолютної помилки та r2-оцінки, можна стверджувати, що поліноміальна модель відповідає вимогам якості, оскільки значення r2-оцінки на тестових даних складає понад 90%, а значення середньої абсолютної помилки менше 10%.


Зокрема, наступні коефіцієнти мають високе значення в поліноміальній моделі:
- Коефіцієнт, що відповідає за квадратичну залежність між ціною та площею квартири. Цей коефіцієнт показує, що збільшення площі квартири відбувається з набагато більшою швидкістю, ніж лінійне збільшення ціни.
- Коефіцієнт, що відповідає за взаємодію між кількістю кімнат та поверхом. Цей коефіцієнт дозволяє враховувати те, що квартири на різних поверхах можуть мати різну ціну, залежно від кількості кімнат.
- Коефіцієнт, що відповідає за взаємодію між роком будівництва та відстанню до центру міста. Цей коефіцієнт показує, що квартири, що знаходяться ближче до центру міста та були побудовані давніше, можуть мати вищу ціну.

## 5. Зберігання побудованх моделей

In [1]:
# зберегти лінийну, гребневу та поліноміальну моделі у відпрвідних pickle-файлах:
# 'lin_model.pkl', 'ridge_model.pkl', 'poly_model.pkl'
import pickle
with open('lin_model.pkl', 'wb') as f:
    pickle.dump(lr, f)
    
with open('ridge_model.pkl', 'wb') as f:
    pickle.dump(ridge, f)
    
with open('poly_model.pkl', 'wb') as f:
    pickle.dump(poly, f)

Ellipsis